In [1]:
!apt-get update
!apt-get install -y colmap

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [966 kB]
Get:14 https

In [2]:
!colmap --help

COLMAP 3.7 -- Structure-from-Motion and Multi-View Stereo
              (Commit Unknown on Unknown without CUDA)

Usage:
  colmap [command] [options]

Documentation:
  https://colmap.github.io/

Example usage:
  colmap help [ -h, --help ]
  colmap gui
  colmap gui -h [ --help ]
  colmap automatic_reconstructor -h [ --help ]
  colmap automatic_reconstructor --image_path IMAGES --workspace_path WORKSPACE
  colmap feature_extractor --image_path IMAGES --database_path DATABASE
  colmap exhaustive_matcher --database_path DATABASE
  colmap mapper --image_path IMAGES --database_path DATABASE --output_path MODEL
  ...

Available commands:
  help
  gui
  automatic_reconstructor
  bundle_adjuster
  color_extractor
  database_cleaner
  database_creator
  database_merger
  delaunay_mesher
  exhaustive_matcher
  feature_extractor
  feature_importer
  hierarchical_mapper
  image_deleter
  image_filterer
  image_rectifier
  image_registrator
  image_undistorter
  image_undistorter_standalone
  mapper

In [4]:

video_path = ''  # Path to the video in Google Drive
frames_folder = 's'  # Path where frames will be saved
reconstruction_output_path = ''  # Path for COLMAP outputs

import os
if not os.path.exists(frames_folder):
    os.makedirs(frames_folder)
if not os.path.exists(reconstruction_output_path):
    os.makedirs(reconstruction_output_path)


In [5]:
import cv2
import os

def extract_frames(video_path, output_folder, interval=5):

    cap = cv2.VideoCapture(video_path)
    count = 0
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save every 'interval' frame
        if frame_idx % interval == 0:
            cv2.imwrite(os.path.join(output_folder, f"frame_{count}.jpg"), frame)
            count += 1
        frame_idx += 1

    cap.release()
    print(f"Extracted {count} frames.")


extract_frames(video_path, frames_folder, interval=5)


Extracted 53 frames.


In [8]:
import os

sparse_output_path = ''#fill this
if os.path.exists(sparse_output_path):
    print("Sparse output path exists:", sparse_output_path)
else:
    print("Sparse output path does not exist")

dense_output_path = '' #to be filled
if os.path.exists(dense_output_path):
    print("Dense output path exists:", dense_output_path)
else:
    print("Dense output path does not exist")

final_model_path = '' #here too
if os.path.exists(final_model_path):
    print("Final model exists:", final_model_path)
else:
    print("Final model does not exist. Ensure COLMAP completed all steps.")


Sparse output path does not exist
Dense output path does not exist
Final model does not exist. Ensure COLMAP completed all steps.


In [6]:
import subprocess


colmap_database_path = os.path.join(reconstruction_output_path, 'database.db')
sparse_output_path = os.path.join(reconstruction_output_path, 'sparse')
dense_output_path = os.path.join(reconstruction_output_path, 'dense')
final_model_path = os.path.join(reconstruction_output_path, 'final_model.ply')

def run_colmap_feature_extraction(image_path, database_path):

    subprocess.run([
        "colmap", "feature_extractor",
        "--database_path", database_path,
        "--image_path", image_path,
        "--ImageReader.single_camera", "1"
    ])

def run_colmap_exhaustive_matcher(database_path):

    subprocess.run([
        "colmap", "exhaustive_matcher",
        "--database_path", database_path
    ])


def run_colmap_mapper(image_path, database_path, output_path):

    subprocess.run([
        "colmap", "mapper",
        "--database_path", database_path,
        "--image_path", image_path,
        "--output_path", output_path
    ])


def run_colmap_image_undistorter(image_path, sparse_path, dense_output_path):

    subprocess.run([
        "colmap", "image_undistorter",
        "--image_path", image_path,
        "--input_path", sparse_path,
        "--output_path", dense_output_path,
        "--output_type", "COLMAP"
    ])


def run_colmap_stereo_fusion(dense_output_path, output_model):

    subprocess.run([
        "colmap", "stereo_fusion",
        "--workspace_path", dense_output_path,
        "--workspace_format", "COLMAP",
        "--output_path", output_model
    ])


run_colmap_feature_extraction(frames_folder, colmap_database_path)
run_colmap_exhaustive_matcher(colmap_database_path)
run_colmap_mapper(frames_folder, colmap_database_path, sparse_output_path)
run_colmap_image_undistorter(frames_folder, sparse_output_path, dense_output_path)
run_colmap_stereo_fusion(dense_output_path, final_model_path)

print("3D reconstruction completed. Final model saved at:", final_model_path)


3D reconstruction completed. Final model saved at: /content/final/final_model.ply


In [ ]:
# Convert PLY to OBJ using MeshlabServer
meshlabserver -i reconstruction_output/final_model.ply -o reconstruction_output/final_model.obj
